# Starter Notebook: GEO AI  Hackathon!


Welcome! This starter notebook is designed to get you started on this challenge, where you will be using time-series Sentinel-2 multi-spectral data to classify crops. We will take a look at the data, create a model and then use that to make our first submission. After that we will briefly look at some ways to improve. Let's get started.

---



---



# IMPORTS

In [2]:
import pandas as pd
import numpy as np

from sklearn.metrics import log_loss , accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

import warnings
warnings.simplefilter('ignore')

In [3]:
from sklearn.linear_model import LogisticRegression, RidgeClassifierCV
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import (RandomForestClassifier, GradientBoostingClassifier)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score

In [4]:
import os
import gc
import random
import pandas as pd
import numpy as np
import lightgbm as lgb

from tqdm.notebook import tqdm
from tqdm import tqdm_notebook

from sklearn.model_selection import GroupKFold
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder


# Loading the Data

We're using the pandas library to load the data into dataframes - a tabular data structure that is perfect for this kind of work. Each of the three CSV files from Zindi is loaded into a dataframe and we take a look at the shape of the data (number of rows and columns) as well as a preview of the first 5 rows to get a feel for what we're working with.

---




* For those who are not familiar with pandas , i highly recommend watching this [video](https://youtu.be/fwWCw_cE5aI) ✌

In [5]:
train = pd.read_csv('Train.csv')
print(train.shape)
train.head()

(1004, 1202)


,ID,Target,timestep1_B02_mean,timestep1_B03_mean,timestep1_B04_mean,timestep1_B05_mean,timestep1_B06_mean,timestep1_B07_mean,timestep1_B08_mean,timestep1_B8A_mean,...,timestep24_B02_min,timestep24_B03_min,timestep24_B04_min,timestep24_B05_min,timestep24_B06_min,timestep24_B07_min,timestep24_B08_min,timestep24_B8A_min,timestep24_B11_min,timestep24_B12_min
0,6fba03cb,olive+cereals,456.918182,799.361364,825.761364,1420.606818,2478.922727,2882.245455,3084.704545,1989.954545,...,113.0,562.0,206.0,1193.0,1497.0,1588.0,1644.0,2060.0,1157.0,1630.0
1,7ea60a74,plowing_and_sowing,1087.130208,1653.770833,2194.458333,2447.984375,2588.328125,2725.609375,2777.385417,3698.088542,...,328.0,913.0,746.0,1468.0,1523.0,1603.0,1782.0,2308.0,1429.0,1741.0
2,64d18595,olive,917.369369,1383.882883,1742.765766,2012.234234,2388.657658,2592.783784,2644.738739,3111.837838,...,113.0,587.0,312.0,1161.0,2050.0,2182.0,2238.0,2016.0,1134.0,2372.0
3,119c8ec4,arable_soil,1013.775120,1551.863636,2073.009569,2371.222488,2666.064593,2843.196172,2930.330144,3802.370813,...,212.0,592.0,810.0,1289.0,1655.0,1796.0,1824.0,2418.0,1708.0,1965.0
4,d884d98c,olive,801.352113,1243.741784,1557.410798,1827.133803,2248.718310,2433.793427,2471.901408,2846.659624,...,84.0,326.0,480.0,790.0,1132.0,1207.0,1194.0,1595.0,1085.0,1283.0


In [6]:
'''%pip install dataprep
from dataprep.datasets import load_dataset
from dataprep.eda import create_report
df = pd.read_csv("Train.csv")
create_report(df).show()'''

'%pip install dataprep\nfrom dataprep.datasets import load_dataset\nfrom dataprep.eda import create_report\ndf = pd.read_csv("Train.csv")\ncreate_report(df).show()'

In [7]:
train=pd.read_csv('Train.csv')
test=pd.read_csv('Test.csv')
sample=pd.read_csv('SampleSubmission.csv')
list_train= train.columns
print(list_train)

Index(['ID', 'Target', 'timestep1_B02_mean', 'timestep1_B03_mean',
       'timestep1_B04_mean', 'timestep1_B05_mean', 'timestep1_B06_mean',
       'timestep1_B07_mean', 'timestep1_B08_mean', 'timestep1_B8A_mean',
       ...
       'timestep24_B02_min', 'timestep24_B03_min', 'timestep24_B04_min',
       'timestep24_B05_min', 'timestep24_B06_min', 'timestep24_B07_min',
       'timestep24_B08_min', 'timestep24_B8A_min', 'timestep24_B11_min',
       'timestep24_B12_min'],
      dtype='object', length=1202)


In [8]:
'''for i in list_train :
  if not (i[14:-1] =='mean' or i[15:-1] =='mean') :
     train.drop(labels = i, axis = 1, inplace = True)'''
train = pd.read_csv('Train.csv')

foo = train.iloc[:, :2]

trainx = train.iloc[:, 2:].filter(like='_mean')

train = pd.concat([foo, trainx], axis=1)

train.filter(regex='timestep _')

l = []
for i in range(10, len(train.columns), 10):
    l.append(train.columns[i])
    l.append(train.columns[i+1])
    
 

train.drop(l, axis=1, inplace=True)

train.shape

train.head()


,ID,Target,timestep1_B02_mean,timestep1_B03_mean,timestep1_B04_mean,timestep1_B05_mean,timestep1_B06_mean,timestep1_B07_mean,timestep1_B08_mean,timestep1_B8A_mean,...,timestep23_B08_mean,timestep23_B8A_mean,timestep24_B02_mean,timestep24_B03_mean,timestep24_B04_mean,timestep24_B05_mean,timestep24_B06_mean,timestep24_B07_mean,timestep24_B08_mean,timestep24_B8A_mean
0,6fba03cb,olive+cereals,456.918182,799.361364,825.761364,1420.606818,2478.922727,2882.245455,3084.704545,1989.954545,...,2809.536364,2447.077273,590.368182,1111.515909,1400.540909,1715.302273,2162.463636,2312.918182,2436.863636,2639.679545
1,7ea60a74,plowing_and_sowing,1087.130208,1653.770833,2194.458333,2447.984375,2588.328125,2725.609375,2777.385417,3698.088542,...,3307.052083,2202.869792,575.468750,1126.234375,1338.671875,1768.291667,2585.895833,2765.562500,2878.385417,2618.859375
2,64d18595,olive,917.369369,1383.882883,1742.765766,2012.234234,2388.657658,2592.783784,2644.738739,3111.837838,...,3697.405405,2348.423423,360.063063,895.252252,784.621622,1520.954955,3140.477477,3480.045045,3694.090090,2387.882883
3,119c8ec4,arable_soil,1013.775120,1551.863636,2073.009569,2371.222488,2666.064593,2843.196172,2930.330144,3802.370813,...,2573.497608,3388.679426,827.315789,1422.665072,1855.571770,2271.423445,2721.198565,2860.514354,3074.009569,3484.294258
4,d884d98c,olive,801.352113,1243.741784,1557.410798,1827.133803,2248.718310,2433.793427,2471.901408,2846.659624,...,2202.943662,2174.800469,352.042254,718.793427,959.312207,1171.706573,1586.760563,1789.600939,1904.948357,2046.302817


In train, we have a set of inputs over 24 diffrent Time steps  and our desired output variable, 'Target'. There are 1010 rows - lots of juicy data!

In [9]:
foo = test.iloc[:, :1]

testx = test.iloc[:, 1:].filter(like='_mean')

test = pd.concat([foo, testx], axis=1)

test.filter(regex='timestep _')

l = []
for i in range(9, len(test.columns), 10):
    l.append(test.columns[i])
    l.append(test.columns[i+1])
    
 

test.drop(l, axis=1, inplace=True)

test.shape

(502, 193)

In [10]:

print(train.shape)
train.head()

(1004, 194)


,ID,Target,timestep1_B02_mean,timestep1_B03_mean,timestep1_B04_mean,timestep1_B05_mean,timestep1_B06_mean,timestep1_B07_mean,timestep1_B08_mean,timestep1_B8A_mean,...,timestep23_B08_mean,timestep23_B8A_mean,timestep24_B02_mean,timestep24_B03_mean,timestep24_B04_mean,timestep24_B05_mean,timestep24_B06_mean,timestep24_B07_mean,timestep24_B08_mean,timestep24_B8A_mean
0,6fba03cb,olive+cereals,456.918182,799.361364,825.761364,1420.606818,2478.922727,2882.245455,3084.704545,1989.954545,...,2809.536364,2447.077273,590.368182,1111.515909,1400.540909,1715.302273,2162.463636,2312.918182,2436.863636,2639.679545
1,7ea60a74,plowing_and_sowing,1087.130208,1653.770833,2194.458333,2447.984375,2588.328125,2725.609375,2777.385417,3698.088542,...,3307.052083,2202.869792,575.468750,1126.234375,1338.671875,1768.291667,2585.895833,2765.562500,2878.385417,2618.859375
2,64d18595,olive,917.369369,1383.882883,1742.765766,2012.234234,2388.657658,2592.783784,2644.738739,3111.837838,...,3697.405405,2348.423423,360.063063,895.252252,784.621622,1520.954955,3140.477477,3480.045045,3694.090090,2387.882883
3,119c8ec4,arable_soil,1013.775120,1551.863636,2073.009569,2371.222488,2666.064593,2843.196172,2930.330144,3802.370813,...,2573.497608,3388.679426,827.315789,1422.665072,1855.571770,2271.423445,2721.198565,2860.514354,3074.009569,3484.294258
4,d884d98c,olive,801.352113,1243.741784,1557.410798,1827.133803,2248.718310,2433.793427,2471.901408,2846.659624,...,2202.943662,2174.800469,352.042254,718.793427,959.312207,1171.706573,1586.760563,1789.600939,1904.948357,2046.302817


In [11]:

print(test.shape)
test.head()

(502, 193)


,ID,timestep1_B02_mean,timestep1_B03_mean,timestep1_B04_mean,timestep1_B05_mean,timestep1_B06_mean,timestep1_B07_mean,timestep1_B08_mean,timestep1_B8A_mean,timestep2_B02_mean,...,timestep23_B08_mean,timestep23_B8A_mean,timestep24_B02_mean,timestep24_B03_mean,timestep24_B04_mean,timestep24_B05_mean,timestep24_B06_mean,timestep24_B07_mean,timestep24_B08_mean,timestep24_B8A_mean
0,d8da32b5,901.974359,1255.743590,1561.230769,1891.230769,2340.333333,2568.282051,2679.179487,3381.820513,958.000000,...,2372.871795,2533.102564,638.641026,1023.538462,1295.102564,1558.179487,1976.564103,2160.051282,2317.179487,2741.000000
1,670ad0fb,927.181818,1423.636364,1778.636364,2161.477273,2745.909091,2969.204545,3063.000000,3632.977273,1067.659091,...,3202.136364,3675.613636,933.500000,1527.636364,2018.136364,2323.568182,2633.590909,2805.159091,2997.909091,3582.954545
2,fec40ac9,716.611940,1126.828358,1390.171642,1751.037313,2480.328358,2757.880597,2798.059701,2913.335821,833.417910,...,1935.388060,2600.074627,427.470149,849.335821,1132.320896,1467.246269,1824.835821,1972.589552,2113.462687,2556.164179
3,4f6d4495,565.611940,931.238806,999.813433,1558.462687,2627.276119,2965.738806,3134.194030,2584.865672,579.500000,...,2690.761194,3536.216418,399.253731,764.074627,1036.208955,1265.104478,1530.291045,1681.305970,1834.223881,2662.723881
4,e56d2db7,943.473430,1371.809179,1709.190821,1926.652174,2180.309179,2325.961353,2378.082126,2846.335749,1064.591787,...,2814.111111,2440.683575,638.731884,1153.584541,1400.545894,1741.442029,2254.026570,2400.330918,2522.874396,2693.857488


In [12]:
for i in range(1,25):
  ttt = train.filter(like='timestep'+str(i)+'_')
  train['train_ndvi_'+str(i)] = (ttt['timestep'+str(i)+'_B8A_mean'] - ttt['timestep'+str(i)+'_B04_mean']) / (ttt['timestep'+str(i)+'_B8A_mean'] + ttt['timestep'+str(i)+'_B04_mean'])


In [13]:
for i in range(1,25):
  ttt = train.filter(like='timestep'+str(i)+'_')
  train['train_evi_'+str(i)] = \
    2.5*(ttt['timestep'+str(i)+'_B8A_mean'] - ttt['timestep'+str(i)+'_B04_mean']) / \
    (ttt['timestep'+str(i)+'_B8A_mean'] + 6*ttt['timestep'+str(i)+'_B04_mean'] - 7.5*ttt['timestep'+str(i)+'_B02_mean']+1)

In [14]:
for i in range(1,25):
  ttt = test.filter(like='timestep'+str(i)+'_')
  test['train_ndvi_'+str(i)] = (ttt['timestep'+str(i)+'_B8A_mean'] - ttt['timestep'+str(i)+'_B04_mean']) / (ttt['timestep'+str(i)+'_B8A_mean'] + ttt['timestep'+str(i)+'_B04_mean'])

In [15]:

for i in range(1,25):
  ttt = test.filter(like='timestep'+str(i)+'_')
  test['train_evi_'+str(i)] = \
    2.5*(ttt['timestep'+str(i)+'_B8A_mean'] - ttt['timestep'+str(i)+'_B04_mean']) / \
    (ttt['timestep'+str(i)+'_B8A_mean'] + 6*ttt['timestep'+str(i)+'_B04_mean'] - 7.5*ttt['timestep'+str(i)+'_B02_mean']+1)


In [16]:
# add 24 gndvi

for i in range(1,25):
  ttt = train.filter(like='timestep'+str(i)+'_')
  train['train_gndvi_'+str(i)] = (ttt['timestep'+str(i)+'_B8A_mean'] - ttt['timestep'+str(i)+'_B03_mean']) / (ttt['timestep'+str(i)+'_B8A_mean'] + ttt['timestep'+str(i)+'_B03_mean'])

for i in range(1,25):
  ttt = test.filter(like='timestep'+str(i)+'_')
  test['train_gndvi_'+str(i)] = (ttt['timestep'+str(i)+'_B8A_mean'] - ttt['timestep'+str(i)+'_B03_mean']) / (ttt['timestep'+str(i)+'_B8A_mean'] + ttt['timestep'+str(i)+'_B03_mean'])

In [17]:
# add 24 cvi

'''for i in range(1,25):
  ttt = train.filter(like='timestep'+str(i)+'_')
  train['train_cvi_'+str(i)] = (ttt['timestep'+str(i)+'_B8A_mean'] / ttt['timestep'+str(i)+'_B03_mean']) * (ttt['timestep'+str(i)+'_B04_mean'] / ttt['timestep'+str(i)+'_B03_mean'])

for i in range(1,25):
  ttt = test.filter(like='timestep'+str(i)+'_')
  test['train_cvi_'+str(i)] = (ttt['timestep'+str(i)+'_B8A_mean'] / ttt['timestep'+str(i)+'_B03_mean']) * (ttt['timestep'+str(i)+'_B04_mean'] / ttt['timestep'+str(i)+'_B03_mean']) '''

"for i in range(1,25):\n  ttt = train.filter(like='timestep'+str(i)+'_')\n  train['train_cvi_'+str(i)] = (ttt['timestep'+str(i)+'_B8A_mean'] / ttt['timestep'+str(i)+'_B03_mean']) * (ttt['timestep'+str(i)+'_B04_mean'] / ttt['timestep'+str(i)+'_B03_mean'])\n\nfor i in range(1,25):\n  ttt = test.filter(like='timestep'+str(i)+'_')\n  test['train_cvi_'+str(i)] = (ttt['timestep'+str(i)+'_B8A_mean'] / ttt['timestep'+str(i)+'_B03_mean']) * (ttt['timestep'+str(i)+'_B04_mean'] / ttt['timestep'+str(i)+'_B03_mean']) "

In [18]:
'''to_delete_train = train.filter(like='timestep')
train.drop(to_delete_train, axis=1, inplace=True)

to_delete_test = test.filter(like='timestep')
test.drop(to_delete_test, axis=1, inplace=True)'''

"to_delete_train = train.filter(like='timestep')\ntrain.drop(to_delete_train, axis=1, inplace=True)\n\nto_delete_test = test.filter(like='timestep')\ntest.drop(to_delete_test, axis=1, inplace=True)"

In [19]:
to_delete_B02_train = train.filter(like='_B02_')

train.drop(to_delete_B02_train, axis=1, inplace=True)

# delete B04
to_delete_B04_train = train.filter(like='_B04_')

train.drop(to_delete_B04_train, axis=1, inplace=True)


In [20]:

to_delete_B02_test = test.filter(like='_B02_')

test.drop(to_delete_B02_test, axis=1, inplace=True)

# delete B04
to_delete_B04_test = test.filter(like='_B04_')

test.drop(to_delete_B04_test, axis=1, inplace=True)


In [21]:
'''to_delete_train = train.filter(like='timestep')
train.drop(to_delete_train, axis=1, inplace=True)

to_delete_test = test.filter(like='timestep')
test.drop(to_delete_test, axis=1, inplace=True)'''

"to_delete_train = train.filter(like='timestep')\ntrain.drop(to_delete_train, axis=1, inplace=True)\n\nto_delete_test = test.filter(like='timestep')\ntest.drop(to_delete_test, axis=1, inplace=True)"

In [22]:
print("Shape of Train Dataset: "+str(train.shape))
print("Shape of Test Dataset: "+str(test.shape))

Shape of Train Dataset: (1004, 218)
Shape of Test Dataset: (502, 217)


In [23]:
from sklearn.preprocessing import StandardScaler




scaler = StandardScaler()
to_scale = train.drop(['ID', 'Target'], axis=1).columns
scaler.fit(train[to_scale])

train[to_scale] = scaler.transform(train[to_scale])


train.head()

,ID,Target,timestep1_B03_mean,timestep1_B05_mean,timestep1_B06_mean,timestep1_B07_mean,timestep1_B08_mean,timestep1_B8A_mean,timestep2_B03_mean,timestep2_B05_mean,...,train_gndvi_15,train_gndvi_16,train_gndvi_17,train_gndvi_18,train_gndvi_19,train_gndvi_20,train_gndvi_21,train_gndvi_22,train_gndvi_23,train_gndvi_24
0,6fba03cb,olive+cereals,-1.546074,-1.325578,-0.381147,0.014539,0.259416,-1.705205,-1.548713,-1.430574,...,-0.048659,0.695822,0.332477,0.726512,1.023069,0.223646,-0.466396,-0.616124,-0.442506,-0.753486
1,7ea60a74,plowing_and_sowing,1.291922,1.302455,-0.038543,-0.414090,-0.546932,0.899003,0.471454,0.538892,...,-0.001719,-0.435941,-0.732158,-0.614645,-0.480326,-0.743845,-0.148895,-0.423169,-0.138950,-0.913962
2,64d18595,olive,0.395466,0.187806,-0.663812,-0.777562,-0.894972,0.005210,0.724275,0.519030,...,0.186965,-0.453478,0.444286,0.500757,0.535410,0.703353,0.322654,-0.099988,0.433834,0.105750
3,119c8ec4,arable_soil,0.953429,1.106098,0.204889,-0.092318,-0.145633,1.057991,0.878272,0.990618,...,0.311004,-0.207234,-0.166312,-0.209998,-0.335480,0.011473,0.139568,0.886098,0.280116,-0.521409
4,d884d98c,olive,-0.070025,-0.285682,-1.102033,-1.212633,-1.348465,-0.399079,0.139900,-0.029512,...,-0.688020,-0.161932,-0.303162,-0.299889,-0.374700,-0.248346,-0.353900,-0.494948,-0.440407,0.568675


In [24]:
from sklearn.preprocessing import StandardScaler




scaler = StandardScaler()
to_scale = test.drop(['ID'], axis=1).columns
scaler.fit(test[to_scale])

test[to_scale] = scaler.transform(test[to_scale])


test.head()

,ID,timestep1_B03_mean,timestep1_B05_mean,timestep1_B06_mean,timestep1_B07_mean,timestep1_B08_mean,timestep1_B8A_mean,timestep2_B03_mean,timestep2_B05_mean,timestep2_B06_mean,...,train_gndvi_15,train_gndvi_16,train_gndvi_17,train_gndvi_18,train_gndvi_19,train_gndvi_20,train_gndvi_21,train_gndvi_22,train_gndvi_23,train_gndvi_24
0,d8da32b5,-0.023479,-0.110513,-0.803279,-0.842370,-0.812003,0.405502,-0.195647,-0.193839,-0.977926,...,0.397645,0.449849,0.429896,0.226519,0.110855,0.339561,0.117007,0.416674,0.228050,0.172679
1,670ad0fb,0.533169,0.582109,0.488160,0.288676,0.240270,0.784972,0.584255,0.632688,0.604762,...,0.743746,0.149823,1.138840,1.157341,-0.547745,-0.555530,-0.678783,-0.277941,-0.821011,-0.849047
2,fec40ac9,-0.450897,-0.469818,-0.357505,-0.307492,-0.486084,-0.302326,-0.215269,-0.231501,-0.220795,...,-0.155307,-0.556306,-0.434524,-0.439258,-0.526365,0.645419,1.060666,2.163742,1.612149,1.022845
3,4f6d4495,-1.099373,-0.963373,0.110408,0.278899,0.435454,-0.798608,-1.160629,-1.078733,0.111886,...,1.295873,0.868275,0.986497,0.740774,0.356948,0.387014,0.254437,1.361042,0.576500,2.022059
4,e56d2db7,0.361337,-0.019730,-1.312830,-1.525983,-1.637484,-0.403556,0.385409,0.077240,-1.417705,...,-0.154695,-0.137106,-0.450918,-0.472464,-0.495963,-0.603431,-0.539930,-1.065302,-0.305547,-0.883667


In [25]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502 entries, 0 to 501
Columns: 217 entries, ID to train_gndvi_24
dtypes: float64(216), object(1)
memory usage: 851.2+ KB


Test looks just like train but without the 'Target' column and with fewer rows.

In [26]:
ss = pd.read_csv('SampleSubmission.csv')
print(ss.shape)
ss.head()

(502, 14)


,ID,arable_soil,cereals,forage_crop,greenhouses,mixed_crops,ochards,olive,olive+arbo,olive+cereals,olive+crops,plowing_and_sowing,vegetable_and_flower,wheat
0,d8da32b5,0,0,0,0,0,0,0,0,1,0,0,0,0
1,670ad0fb,0,0,0,0,0,0,0,0,0,0,1,0,0
2,fec40ac9,0,0,0,0,0,0,1,0,0,0,0,0,0
3,4f6d4495,1,0,0,0,0,0,0,0,0,0,0,0,0
4,e56d2db7,0,0,0,0,0,0,0,1,0,0,0,0,0


# Data Processing

---



In [27]:
 date_dict={1: '2021-04-28', 2: '2021-04-25', 3: '2021-03-06', 4: '2021-02-19', 5: '2021-02-12', 6: '2021-01-23', 7: '2021-01-15', 8: '2020-12-31', 9: '2020-12-29', 10: '2020-12-24', 11: '2020-12-16', 12: '2020-11-26', 13: '2020-11-14', 14: '2020-11-09', 15: '2021-03-29', 16: '2021-03-11', 17: '2021-02-27', 18: '2021-02-17', 19: '2021-02-09', 20: '2021-01-30', 21: '2021-01-20', 22: '2021-01-08', 23: '2021-01-13', 24: '2020-12-14'}

In [28]:
''' from scipy.stats import boxcox 
#perform Box-Cox transformation on original data
train['CIF_USD_EQUIVALENT_boxcox'], best_lambda = boxcox(train['CIF_USD_EQUIVALENT'])
train['GROSS.WEIGHT_boxcox'], best_lambda = boxcox(train['GROSS.WEIGHT'])
#train['OFFICE_boxcox'], best_lambda = boxcox(train['OFFICE'])
#train['IMPORTER.TIN_boxcox'], best_lambda = boxcox(train['IMPORTER.TIN'])
train['TARIFF.CODE_boxcox'], best_lambda = boxcox(train['TARIFF.CODE'])
#train['ORIGIN.CODE_boxcox'], best_lambda = boxcox(train['ORIGIN.CODE'])
train['QUANTITY_boxcox'], best_lambda = boxcox(train['QUANTITY'])
#train['TOTAL.TAXES.USD_boxcox'], best_lambda = boxcox(train['TOTAL.TAXES.USD'])

test['CIF_USD_EQUIVALENT_boxcox'], best_lambda = boxcox(test['CIF_USD_EQUIVALENT'])
test['GROSS.WEIGHT_boxcox'], best_lambda = boxcox(test['GROSS.WEIGHT'])
#test['OFFICE_boxcox'], best_lambda = boxcox(test['OFFICE'])
#test['IMPORTER.TIN_boxcox'], best_lambda = boxcox(test['IMPORTER.TIN'])
test['TARIFF.CODE_boxcox'], best_lambda = boxcox(test['TARIFF.CODE'])
#test['ORIGIN.CODE_boxcox'], best_lambda = boxcox(test['ORIGIN.CODE'])
test['QUANTITY_boxcox'], best_lambda = boxcox(test['QUANTITY'])
#test['TOTAL.TAXES.USD_boxcox'], best_lambda = boxcox(test['TOTAL.TAXES.USD']) '''

" from scipy.stats import boxcox \n#perform Box-Cox transformation on original data\ntrain['CIF_USD_EQUIVALENT_boxcox'], best_lambda = boxcox(train['CIF_USD_EQUIVALENT'])\ntrain['GROSS.WEIGHT_boxcox'], best_lambda = boxcox(train['GROSS.WEIGHT'])\n#train['OFFICE_boxcox'], best_lambda = boxcox(train['OFFICE'])\n#train['IMPORTER.TIN_boxcox'], best_lambda = boxcox(train['IMPORTER.TIN'])\ntrain['TARIFF.CODE_boxcox'], best_lambda = boxcox(train['TARIFF.CODE'])\n#train['ORIGIN.CODE_boxcox'], best_lambda = boxcox(train['ORIGIN.CODE'])\ntrain['QUANTITY_boxcox'], best_lambda = boxcox(train['QUANTITY'])\n#train['TOTAL.TAXES.USD_boxcox'], best_lambda = boxcox(train['TOTAL.TAXES.USD'])\n\ntest['CIF_USD_EQUIVALENT_boxcox'], best_lambda = boxcox(test['CIF_USD_EQUIVALENT'])\ntest['GROSS.WEIGHT_boxcox'], best_lambda = boxcox(test['GROSS.WEIGHT'])\n#test['OFFICE_boxcox'], best_lambda = boxcox(test['OFFICE'])\n#test['IMPORTER.TIN_boxcox'], best_lambda = boxcox(test['IMPORTER.TIN'])\ntest['TARIFF.CODE_box

In [29]:
'''# train
cor_matrix = train.corr().abs()



upper_tri = cor_matrix.where(np.triu(np.ones(cor_matrix.shape),k=1).astype(np.bool))


to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.95)]

print(len(to_drop))

train.drop(to_drop, axis=1, inplace=True)


# test


test.drop(to_drop, axis=1, inplace=True)
print(train.shape)
print(test.shape)'''

'# train\ncor_matrix = train.corr().abs()\n\n\n\nupper_tri = cor_matrix.where(np.triu(np.ones(cor_matrix.shape),k=1).astype(np.bool))\n\n\nto_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.95)]\n\nprint(len(to_drop))\n\ntrain.drop(to_drop, axis=1, inplace=True)\n\n\n# test\n\n\ntest.drop(to_drop, axis=1, inplace=True)\nprint(train.shape)\nprint(test.shape)'

In [30]:
def process(train,test,ss) :
  
  target_mapper = dict(zip(ss.drop('ID',1).columns.tolist(),
                          [i for i in range(len(ss.drop('ID',1).columns.tolist()))]))  # Used to Encode Train Target 
  train['Target'] = train['Target'].map(target_mapper)
  
  Inversetarget_mapper = dict(zip([i for i in range(len(ss.drop('ID',1).columns.tolist()))],
                                  ss.drop('ID',1).columns.tolist())) # Used to Create submission file 

  in_cols = train.iloc[:,2:].columns.tolist() # features used in Training, we will use only time step 1 in this tutorial

  return target_mapper , Inversetarget_mapper , in_cols , train , test , ss

In [31]:
#train = pd.read_csv('Train.csv')
#test = pd.read_csv('Test.csv')
#ss = pd.read_csv('SampleSubmission.csv')

target_mapper , Inversetarget_mapper , in_cols , train , test , ss  = process(train,test,ss)
print(in_cols)

['timestep1_B03_mean', 'timestep1_B05_mean', 'timestep1_B06_mean', 'timestep1_B07_mean', 'timestep1_B08_mean', 'timestep1_B8A_mean', 'timestep2_B03_mean', 'timestep2_B05_mean', 'timestep2_B06_mean', 'timestep2_B07_mean', 'timestep2_B08_mean', 'timestep2_B8A_mean', 'timestep3_B03_mean', 'timestep3_B05_mean', 'timestep3_B06_mean', 'timestep3_B07_mean', 'timestep3_B08_mean', 'timestep3_B8A_mean', 'timestep4_B03_mean', 'timestep4_B05_mean', 'timestep4_B06_mean', 'timestep4_B07_mean', 'timestep4_B08_mean', 'timestep4_B8A_mean', 'timestep5_B03_mean', 'timestep5_B05_mean', 'timestep5_B06_mean', 'timestep5_B07_mean', 'timestep5_B08_mean', 'timestep5_B8A_mean', 'timestep6_B03_mean', 'timestep6_B05_mean', 'timestep6_B06_mean', 'timestep6_B07_mean', 'timestep6_B08_mean', 'timestep6_B8A_mean', 'timestep7_B03_mean', 'timestep7_B05_mean', 'timestep7_B06_mean', 'timestep7_B07_mean', 'timestep7_B08_mean', 'timestep7_B8A_mean', 'timestep8_B03_mean', 'timestep8_B05_mean', 'timestep8_B06_mean', 'timestep

# MODELING

---



In [32]:
X, y = train[in_cols], train['Target']
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2,stratify = y ,
                                                    random_state=58) # Random state keeps the split consistent
print(X_train.shape, X_test.shape)

(803, 216) (201, 216)


In [33]:
'''model = RandomForestClassifier(random_state=0) # Create the model
model.fit(X_train, y_train) # Train it (this syntax looks the same for all sklearn models)random_forest
print('LOCAL log_loss',log_loss(y_test, model.predict_proba(X_test))) # predict_proba return probabilities 
print('LOCAL accuracy',accuracy_score(y_test, model.predict(X_test)))'''

"model = RandomForestClassifier(random_state=0) # Create the model\nmodel.fit(X_train, y_train) # Train it (this syntax looks the same for all sklearn models)random_forest\nprint('LOCAL log_loss',log_loss(y_test, model.predict_proba(X_test))) # predict_proba return probabilities \nprint('LOCAL accuracy',accuracy_score(y_test, model.predict(X_test)))"

In [34]:
# Random Forest

random_forest = RandomForestClassifier(criterion= 'entropy', max_features= 'auto', min_samples_leaf= 3, n_estimators= 200)
random_forest.fit(X_train, y_train)
print('LOCAL log_loss',log_loss(y_test, random_forest.predict_proba(X_test))) # predict_proba return probabilities 
print('LOCAL accuracy',accuracy_score(y_test, random_forest.predict(X_test)))

LOCAL log_loss 1.7395014031306715
LOCAL accuracy 0.4925373134328358


In [35]:
# Gaussian Naive Bayes
gaussian = GaussianNB()
gaussian.fit(X_train, y_train)
print('LOCAL log_loss',log_loss(y_test, gaussian.predict_proba(X_test))) # predict_proba return probabilities 
print('LOCAL accuracy',accuracy_score(y_test, gaussian.predict(X_test)))

LOCAL log_loss 18.987355075862812
LOCAL accuracy 0.2736318407960199


In [36]:
# Logistic Regression
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
print('LOCAL log_loss',log_loss(y_test, logreg.predict_proba(X_test))) # predict_proba return probabilities 
print('LOCAL accuracy',accuracy_score(y_test, logreg.predict(X_test)))

LOCAL log_loss 1.8419078809417
LOCAL accuracy 0.5174129353233831


In [37]:
'''# Support Vector Machines
svc = SVC()
svc.fit(X_train, y_train)
Y_pred = svc.predict(X_test)
acc_svc = round(svc.score(X_train, y_train) * 100, 2)
acc_svc'''

'# Support Vector Machines\nsvc = SVC()\nsvc.fit(X_train, y_train)\nY_pred = svc.predict(X_test)\nacc_svc = round(svc.score(X_train, y_train) * 100, 2)\nacc_svc'

In [38]:
'''knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train, y_train)
print('LOCAL log_loss',log_loss(y_test, knn.predict_proba(X_test))) # predict_proba return probabilities 
print('LOCAL accuracy',accuracy_score(y_test, knn.predict(X_test)))'''

"knn = KNeighborsClassifier(n_neighbors = 3)\nknn.fit(X_train, y_train)\nprint('LOCAL log_loss',log_loss(y_test, knn.predict_proba(X_test))) # predict_proba return probabilities \nprint('LOCAL accuracy',accuracy_score(y_test, knn.predict(X_test)))"

In [39]:
'''# Perceptron
perceptron = Perceptron()
perceptron.fit(X_train, y_train)
print('LOCAL log_loss',log_loss(y_test, gaussian.predict_proba(X_test))) # predict_proba return probabilities 
print('LOCAL accuracy',accuracy_score(y_test, gaussian.predict(X_test)))'''

"# Perceptron\nperceptron = Perceptron()\nperceptron.fit(X_train, y_train)\nprint('LOCAL log_loss',log_loss(y_test, gaussian.predict_proba(X_test))) # predict_proba return probabilities \nprint('LOCAL accuracy',accuracy_score(y_test, gaussian.predict(X_test)))"

In [40]:
'''# Decision Tree
decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, y_train)
print('LOCAL log_loss',log_loss(y_test, decision_tree.predict_proba(X_test))) # predict_proba return probabilities 
print('LOCAL accuracy',accuracy_score(y_test, decision_tree.predict(X_test)))'''

"# Decision Tree\ndecision_tree = DecisionTreeClassifier()\ndecision_tree.fit(X_train, y_train)\nprint('LOCAL log_loss',log_loss(y_test, decision_tree.predict_proba(X_test))) # predict_proba return probabilities \nprint('LOCAL accuracy',accuracy_score(y_test, decision_tree.predict(X_test)))"

In [41]:
'''# Gradient Boosting
grad_boost = GradientBoostingClassifier(n_estimators = 100)
grad_boost.fit(X_train, y_train)
print('LOCAL log_loss',log_loss(y_test, grad_boost.predict_proba(X_test))) # predict_proba return probabilities 
print('LOCAL accuracy',accuracy_score(y_test, grad_boost.predict(X_test)))'''

"# Gradient Boosting\ngrad_boost = GradientBoostingClassifier(n_estimators = 100)\ngrad_boost.fit(X_train, y_train)\nprint('LOCAL log_loss',log_loss(y_test, grad_boost.predict_proba(X_test))) # predict_proba return probabilities \nprint('LOCAL accuracy',accuracy_score(y_test, grad_boost.predict(X_test)))"

In [42]:
%pip install catboost
from catboost import CatBoostRegressor
cb = CatBoostRegressor(n_estimators=100,
                       loss_function='RMSE',
                       learning_rate=0.1,
                       depth=3, task_type='CPU',
                       random_state=1,
                       verbose=False)

cb.fit(X_train, y_train)
y_pred = cb.predict(X_test)

In [43]:
'''# RidgeClassifierCV
Ridge= RidgeClassifierCV()
Ridge.fit(X_train, y_train)
Y_pred = Ridge.predict(X_test)
acc_Ridge= round(Ridge.score(X_train, y_train) * 100, 2)
acc_Ridge'''

'# RidgeClassifierCV\nRidge= RidgeClassifierCV()\nRidge.fit(X_train, y_train)\nY_pred = Ridge.predict(X_test)\nacc_Ridge= round(Ridge.score(X_train, y_train) * 100, 2)\nacc_Ridge'

In [44]:
# Split the training set into a development and an evaluation sets
from sklearn.model_selection import train_test_split
X_dev, X_eval, y_dev, y_eval = train_test_split(X_train,
                                                y_train,
                                                test_size=0.2,
                                                random_state=42)

X_test.head(10)

,timestep1_B03_mean,timestep1_B05_mean,timestep1_B06_mean,timestep1_B07_mean,timestep1_B08_mean,timestep1_B8A_mean,timestep2_B03_mean,timestep2_B05_mean,timestep2_B06_mean,timestep2_B07_mean,...,train_gndvi_15,train_gndvi_16,train_gndvi_17,train_gndvi_18,train_gndvi_19,train_gndvi_20,train_gndvi_21,train_gndvi_22,train_gndvi_23,train_gndvi_24
129,1.013369,1.171541,0.331108,0.006389,-0.164957,1.066330,1.197361,1.338726,0.375652,-0.023300,...,-0.477055,-0.468601,-0.092233,-0.365860,-0.523956,-0.844835,-0.411713,0.170298,-0.082429,-0.531448
579,0.781763,0.641880,-0.004669,-0.253958,-0.348295,0.530444,0.657718,0.501216,-0.178889,-0.401726,...,-1.859134,-0.796123,-1.907022,-1.619334,-1.022764,-1.105743,-1.354021,-1.608014,-1.109971,-1.343184
60,1.123061,1.019623,-0.042511,-0.369785,-0.553045,0.899495,1.171018,1.073885,-0.113493,-0.472131,...,1.262765,-0.867169,-0.364275,-0.501194,-0.650635,0.004525,0.142110,1.312074,0.553338,-1.771576
341,-0.650205,-0.549572,1.585499,2.151520,2.203220,-0.794048,-1.003627,-0.966266,1.628494,2.317887,...,1.559308,0.597224,-0.836062,-0.688551,-0.850597,-1.281302,-0.356122,0.509296,-0.247297,-1.054401
560,-1.734580,-1.924850,-0.979251,-0.218687,-0.055114,-1.801842,-1.742293,-1.895258,-0.934020,0.028305,...,1.765596,2.281311,1.339547,1.483999,1.312176,0.378333,-0.086968,-0.624152,-0.518611,-0.738646
974,1.949992,1.855175,2.007504,1.625203,1.502011,1.608191,1.842547,1.741266,1.973317,1.577374,...,-0.998765,-0.704738,-0.270053,-0.522333,-0.737876,-0.659053,-0.921689,-0.422611,-0.864021,-0.459669
337,1.182677,1.119601,1.335477,1.093885,0.973824,1.214033,1.183436,1.156666,1.306858,1.012408,...,-0.038599,-0.368426,0.731714,0.692435,0.756317,1.115377,1.083507,1.480329,1.487190,1.453771
35,0.673706,0.522177,-0.016163,-0.204913,-0.256634,0.489706,0.715166,0.617372,0.036631,-0.178256,...,-0.734867,-0.578990,-0.595187,-0.540362,-0.373768,-0.278486,-0.981383,-1.039990,-0.794879,-0.869070
904,0.977984,1.056992,1.263151,1.014016,0.946326,1.204983,1.040217,1.126913,1.188283,0.902287,...,0.314118,-0.021945,0.721427,0.576526,0.457034,0.706268,0.551107,1.204813,0.627100,0.866531
986,0.954292,1.138479,1.396863,1.287732,1.254862,1.165334,0.890792,1.074042,1.408017,1.261737,...,0.820188,-0.433432,-0.342635,-0.459942,-0.522375,-0.375942,-0.322814,-0.163489,-0.052085,-0.297669


In [45]:
'''# 1. Random Forest
import time
dict_clf = {}


paramgrid = {
    'n_estimators':      [50,75, 125, 175, 225],
    'criterion':         ['gini', 'entropy'],
    'max_features':      ['auto', 'log2'],
    'min_samples_leaf':  list(range(2, 8))
}
GS = GridSearchCV(RandomForestClassifier(random_state=77),
                  paramgrid,
                  cv=4)

# Fit the data and record time taking to train
t0 = time.time()
GS.fit(X_dev, y_dev)
t = time.time() - t0

# Store best parameters, score and estimator
best_clf = GS.best_estimator_
best_params = GS.best_params_
best_score = GS.best_score_

name = 'RF'    '''

"# 1. Random Forest\nimport time\ndict_clf = {}\n\n\nparamgrid = {\n    'n_estimators':      [50,75, 125, 175, 225],\n    'criterion':         ['gini', 'entropy'],\n    'max_features':      ['auto', 'log2'],\n    'min_samples_leaf':  list(range(2, 8))\n}\nGS = GridSearchCV(RandomForestClassifier(random_state=77),\n                  paramgrid,\n                  cv=4)\n\n# Fit the data and record time taking to train\nt0 = time.time()\nGS.fit(X_dev, y_dev)\nt = time.time() - t0\n\n# Store best parameters, score and estimator\nbest_clf = GS.best_estimator_\nbest_params = GS.best_params_\nbest_score = GS.best_score_\n\nname = 'RF'    "

In [46]:
'''print(best_params)'''

'print(best_params)'

In [47]:
'''best_clf.fit(X_dev, y_dev)
acc_eval = accuracy_score(y_eval, best_clf.predict(X_eval))

dict_clf[name] = {
    'best_par': best_params,
    'best_clf': best_clf,
    'best_score': best_score,
    'score_eval': acc_eval,
    'fit_time': t,
}

acc_eval'''

"best_clf.fit(X_dev, y_dev)\nacc_eval = accuracy_score(y_eval, best_clf.predict(X_eval))\n\ndict_clf[name] = {\n    'best_par': best_params,\n    'best_clf': best_clf,\n    'best_score': best_score,\n    'score_eval': acc_eval,\n    'fit_time': t,\n}\n\nacc_eval"

In [48]:
'''from sklearn.ensemble import VotingClassifier

estimators = [('RF', dict_clf['RF']['best_clf'])]

              

# Instantiate the VotingClassifier using hard voting
voter = VotingClassifier(estimators=estimators, voting='hard')
voter.fit(X_train, y_train)

Y_pred = voter.predict(X_test).astype(int)'''

"from sklearn.ensemble import VotingClassifier\n\nestimators = [('RF', dict_clf['RF']['best_clf'])]\n\n              \n\n# Instantiate the VotingClassifier using hard voting\nvoter = VotingClassifier(estimators=estimators, voting='hard')\nvoter.fit(X_train, y_train)\n\nY_pred = voter.predict(X_test).astype(int)"

# SUBMISSION

---



In [49]:
test_prediction = random_forest.predict_proba(test[in_cols])

sub = pd.DataFrame(test_prediction)
sub = sub.rename(columns=Inversetarget_mapper)
sub['ID'] =  test['ID'].values
submission = pd.read_csv('SampleSubmission.csv')
submission = pd.merge(sub[['ID']],submission,on='ID',how='left')

for col in submission.columns[1:] :
  submission[col] = sub[col]
  
submission.head()

,ID,arable_soil,cereals,forage_crop,greenhouses,mixed_crops,ochards,olive,olive+arbo,olive+cereals,olive+crops,plowing_and_sowing,vegetable_and_flower,wheat
0,d8da32b5,0.059625,0.028885,0.022312,0.007583,0.020060,0.143615,0.531226,0.106722,0.044516,0.022667,0.004262,0.003667,0.004861
1,670ad0fb,0.094449,0.024167,0.010804,0.007250,0.066273,0.082268,0.214145,0.127020,0.085376,0.092371,0.117817,0.037208,0.040852
2,fec40ac9,0.016838,0.013804,0.021351,0.002222,0.045694,0.388967,0.245032,0.065086,0.012417,0.105672,0.025798,0.048202,0.008917
3,4f6d4495,0.069185,0.050407,0.023667,0.005556,0.041852,0.237345,0.288518,0.126859,0.086466,0.030550,0.005500,0.006750,0.027347
4,e56d2db7,0.072679,0.275922,0.078816,0.005889,0.028288,0.040095,0.031040,0.010696,0.162458,0.005833,0.060496,0.000000,0.227788


In [50]:
submission.to_csv('submission15.csv',index=False)